# 합성 테스트 데이터셋 생성

**왜 합성 테스트 데이터(Synthetic Test Dataset) 인가?**

RAG(검색 증강 생성) 증강 파이프라인의 성능을 평가하는 것은 매우 중요합니다. 

그러나 문서에서 수백 개의 QA(질문-문맥-응답) 샘플을 수동으로 생성하는 것은 시간과 노동력이 많이 소요될 수 있습니다. 또한 사람이 만든 질문은 철저한 평가에 필요한 복잡성 수준에 도달하기 어려워 궁극적으로 평가의 품질에 영향을 미칠 수 있습니다. 

합성 데이터 생성을 사용하면 데이터 집계 프로세스에서 **개발자의 시간을 90%** 까지 줄일 수 있습니다.

- RAGAS: https://docs.ragas.io/en/latest/concepts/testset_generation.html

In [9]:
import langchain
import ragas

print(f"LangChain Version: {langchain.__version__}")
print(f"Ragas Version: {ragas.__version__}")

LangChain Version: 0.3.21
Ragas Version: 0.2.14


In [2]:
import os
import sys
import json
import time
import requests

from dotenv import load_dotenv

def setup_env():
    
    env_path = os.path.join(os.getcwd(), '../.env')

    if os.path.exists(env_path):
        load_dotenv(dotenv_path=env_path)
        
        print(f"Loaded environment variables from: \033[94m{env_path}\033[0m")
    else:
            print("\033[91mError: .env file not found. Please create one with your OPENAI_API_KEY.\033[0m")
            sys.exit(1)

setup_env()

Loaded environment variables from: /home/ras/0.agent_ai_ws/src/learn_rag_and_agent/learn_rag_and_agent/../.env


In [4]:
from langchain_community.document_loaders import PDFPlumberLoader

# 문서 로더 생성
loader = PDFPlumberLoader("SPRI_AI_Brief_2023년12월호_F.pdf")

# 문서 로딩
docs = loader.load()

# 목차, 끝 페이지 제외
docs = docs[3:-1]

# 문서의 페이지수
len(docs)

19

각 문서 객체에는 `metadata` 를 통해 액세스할 수 있는 문서에 대한 추가 정보를 저장하는 데 사용할 수 있는 메타데이터 사전이 포함되어 있습니다. 

메타데이터 사전에는 `filename` 이라는 키가 포함되어 있는지 확인하세요. 

이 키는 Test datasets 생성 프로세스에서 활용될 것이므로. 메타데이터의 `filename` 속성은 동일한 문서에 속한 청크를 식별하는 데 사용됩니다. 

In [6]:
docs[0].metadata

{'source': 'SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': 'SPRI_AI_Brief_2023년12월호_F.pdf',
 'page': 3,
 'total_pages': 23,
 'Author': 'dj',
 'Creator': 'Hwp 2018 10.0.0.13462',
 'Producer': 'Hancom PDF 1.3.0.542',
 'CreationDate': "D:20231208132838+09'00'",
 'ModDate': "D:20231208132838+09'00'",
 'PDFVersion': '1.4'}

In [7]:
# metadata 설정(filename 이 존재해야 함)
for doc in docs:
    doc.metadata["filename"] = doc.metadata["source"]

In [10]:
from ragas.testset import TestsetGenerator
from ragas.testset.synthesizers import default_query_distribution
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 데이터셋 생성기
generator_llm = ChatOpenAI(model="gpt-4o-mini")

# 데이터셋 비평기
critic_llm = ChatOpenAI(model="gpt-4o-mini")

# 문서 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [12]:
# LangChain의 LLM과 임베딩을 Ragas용 래퍼로 감쌉니다.
generator_llm_wrapper = LangchainLLMWrapper(generator_llm)
critic_llm_wrapper = LangchainLLMWrapper(critic_llm)
embeddings_wrapper = LangchainEmbeddingsWrapper(embeddings)

# TestsetGenerator를 초기화합니다.
testset_generator = TestsetGenerator(
    llm=generator_llm_wrapper,
    embedding_model=embeddings_wrapper
)

In [13]:
# 질문 유형별 분포 결정
# simple: 간단한 질문, reasoning: 추론이 필요한 질문, multi_context: 여러 맥락을 고려해야 하는 질문, conditional: 조건부 질문
distributions = {"simple": 0.4, "reasoning": 0.2, "multi_context": 0.2, "conditional": 0.2}

In [14]:
# 테스트 데이터셋을 생성합니다.
# testset_size: 생성할 질문-답변 쌍의 개수
testset = testset_generator.generate_with_langchain_docs(
    documents=docs,
    testset_size=10,
    query_distribution=default_query_distribution(generator_llm_wrapper)
)

# 생성된 데이터셋 확인
test_df = testset.to_pandas()

Applying HeadlinesExtractor:   0%|          | 0/19 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/19 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node '7d3121'. Skipping!
Property 'summary' already exists in node '0d49a8'. Skipping!
Property 'summary' already exists in node '0d49a8'. Skipping!
Property 'summary' already exists in node '4739ea'. Skipping!
Property 'summary' already exists in node '4739ea'. Skipping!
Property 'summary' already exists in node '97122f'. Skipping!
Property 'summary' already exists in node '97122f'. Skipping!
Property 'summary' already exists in node '79868f'. Skipping!
Property 'summary' already exists in node '79868f'. Skipping!
Property 'summary' already exists in node 'e31e3d'. Skipping!
Property 'summary' already exists in node 'e31e3d'. Skipping!
Property 'summary' already exists in node 'f36a7f'. Skipping!
Property 'summary' already exists in node 'f36a7f'. Skipping!
Property 'summary' already exists in node 'a4e761'. Skipping!
Property 'summary' already exists in node 'a4e761'. Skipping!
Property 'summary' already exists in node 'e5d067'. Skipping!
Property

Applying CustomNodeFilter: 0it [00:00, ?it/s]

...

Property 'summary_embedding' already exists in node '97122f'. Skipping!
Property 'summary_embedding' already exists in node '7d3121'. Skipping!
Property 'summary_embedding' already exists in node '7d3121'. Skipping!
Property 'summary_embedding' already exists in node 'f36a7f'. Skipping!
Property 'summary_embedding' already exists in node 'f36a7f'. Skipping!
Property 'summary_embedding' already exists in node '4739ea'. Skipping!
Property 'summary_embedding' already exists in node '4739ea'. Skipping!
Property 'summary_embedding' already exists in node '0d49a8'. Skipping!
Property 'summary_embedding' already exists in node '0d49a8'. Skipping!
Property 'summary_embedding' already exists in node '79868f'. Skipping!
Property 'summary_embedding' already exists in node '79868f'. Skipping!
Property 'summary_embedding' already exists in node 'e31e3d'. Skipping!
Property 'summary_embedding' already exists in node 'e31e3d'. Skipping!
Property 'summary_embedding' already exists in node 'e87b54'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

ValueError: No clusters found in the knowledge graph. Try changing the relationship condition.

In [ ]:
# 생성된 테스트셋을 pandas DataFrame으로 변환
test_df = testset.to_pandas()
test_df

In [ ]:
# DataFrame의 상위 5개 행 출력
test_df.head()

In [ ]:
# DataFrame을 CSV 파일로 저장
test_df.to_csv("data/ragas_synthetic_dataset.csv", index=False)